In [85]:
import numpy as np
import pandas as pd
import yaml
import torch
from copy import deepcopy
from tqdm import tqdm
import matplotlib.pyplot as plt
from celldreamer.eval.eval_utils import normalize_and_compute_metrics
import scipy.sparse as sp

import muon as mu
from torch import nn
import scanpy as sc    

from celldreamer.data.scrnaseq_loader import RNAseqLoader
from celldreamer.models.featurizers.category_featurizer import CategoricalFeaturizer
from celldreamer.models.fm.fm import FM
from celldreamer.eval.optimal_transport import wasserstein
import random
from celldreamer.models.base.encoder_model import EncoderModel
from celldreamer.models.base.utils import unsqueeze_right

from celldreamer.paths import DATA_DIR

device  = "cuda" if torch.cuda.is_available() else "cpu"

sc.settings.figdir = 'figures'  # Directory to save figures
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(6, 6)) 

## Step 1: Initialize data

In [86]:
dataset_config = {'dataset_path': DATA_DIR / 'processed' / 'atac' / 'neurips' / 'neurips_multiome_test.h5mu',
                    'layer_key': 'X_counts',
                    'covariate_keys': ['cell_type'],
                    'conditioning_covariate': 'cell_type',
                    'subsample_frac': 1,
                    'encoder_type': 'learnt_autoencoder',
                    'one_hot_encode_features': False,
                    'split_rates': [0.90, 0.10],
                    'cov_embedding_dimensions': 256, 
                    'multimodal': True,
                    'is_binarized': False}

data_path = dataset_config["dataset_path"]

dataset = RNAseqLoader(data_path = data_path,
                        layer_key=dataset_config["layer_key"],
                        covariate_keys=dataset_config["covariate_keys"],
                        subsample_frac=dataset_config["subsample_frac"], 
                        encoder_type=dataset_config["encoder_type"], 
                        multimodal=dataset_config["multimodal"],
                        is_binarized=dataset_config["is_binarized"])

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/mudata/_core/mudata.py:577: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_common = pd.concat(


In [87]:
gene_dim = {mod: dataset.X[mod].shape[1] for mod in dataset.X}
modality_list = list(gene_dim.keys())
in_dim = {}

for mod in dataset.X:
    if dataset_config["encoder_type"]!="learnt_autoencoder":
        in_dim[mod] = gene_dim[mod]
    else:
        in_dim[mod] = encoder_config["encoder_kwargs"][mod]["dims"][-1]
                    
size_factor_statistics = {"mean": {mod: dataset.log_size_factor_mu[mod] for mod in dataset.log_size_factor_mu}, 
                            "sd": {mod: dataset.log_size_factor_sd[mod] for mod in dataset.log_size_factor_sd}}

n_cat = len(dataset.id2cov["cell_type"])

## Step 2: Initialize encoder

In [88]:
encoder_config = {"encoder_kwargs": 
                      {"rna": {"dims": [512, 300, 100],
                               "batch_norm": True,
                               "dropout": False,
                               "dropout_p": 0.0},
                       "atac": {"dims": [1024, 512, 100],
                               "batch_norm": True,
                               "dropout": False,
                               "dropout_p": 0.0}},
                    "learning_rate": 0.001,
                    "weight_decay": 0.00001,
                    "covariate_specific_theta": False,
                    "multimodal": True, 
                    "is_binarized": False,
                    "encoder_multimodal_joint_layers":
                        {"dims": [100],
                        "batch_norm": False,
                        "dropout": False, 
                        "dropout_p": 0.0}
                 }

state_dict_path = "/home/icb/alessandro.palma/environment/celldreamer/project_folder/experiments/train_autoencoder_neurips_multimodal_joint/1c26a129-9280-421c-b941-1203cd3887a2/checkpoints/last.ckpt"

In [89]:
encoder_model = EncoderModel(in_dim=gene_dim,
                              n_cat=n_cat,
                              conditioning_covariate=dataset_config["conditioning_covariate"], 
                              encoder_type=dataset_config["encoder_type"],
                              **encoder_config)


encoder_model.load_state_dict(torch.load(state_dict_path)["state_dict"])

encoder_model.eval()

EncoderModel(
  (x0_from_x): ModuleDict(
    (rna): MLP(
      (net): Sequential(
        (0): Sequential(
          (0): Linear(in_features=4000, out_features=512, bias=True)
          (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ELU(alpha=1.0)
        )
        (1): Sequential(
          (0): Linear(in_features=512, out_features=300, bias=True)
          (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ELU(alpha=1.0)
        )
        (2): Linear(in_features=300, out_features=100, bias=True)
      )
    )
    (atac): MLP(
      (net): Sequential(
        (0): Sequential(
          (0): Linear(in_features=40002, out_features=1024, bias=True)
          (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ELU(alpha=1.0)
        )
        (1): Sequential(
          (0): Linear(in_features=1024, out_features=512, bias=True)
          

## Step 3: Initialize FM model

In [90]:
generative_model_config = {'learning_rate': 0.0001,
                            'weight_decay': 0.00001,
                            'antithetic_time_sampling': True,
                            'sigma': 0.0001
                        }

In [91]:
ckpt = torch.load("/home/icb/alessandro.palma/environment/celldreamer/project_folder/experiments/fm_resnet_autoencoder_neurips_multimodal_joint/dd54e62d-d7a1-4c16-b58a-c8543d33c961/checkpoints/last.ckpt")

denoising_model = ckpt["hyper_parameters"]["denoising_model"]
denoising_model

MLPTimeStep(
  (time_embedder): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): SiLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
  )
  (net_in): Linear(in_features=100, out_features=64, bias=True)
  (blocks): ModuleList(
    (0-2): 3 x ResnetBlock(
      (net1): Sequential(
        (0): SiLU()
        (1): Linear(in_features=64, out_features=64, bias=True)
      )
      (cond_proj): Sequential(
        (0): SiLU()
        (1): Linear(in_features=100, out_features=64, bias=True)
      )
      (net2): Sequential(
        (0): SiLU()
        (1): Linear(in_features=64, out_features=64, bias=True)
      )
    )
  )
  (net_out): Sequential(
    (0): SiLU()
    (1): Linear(in_features=64, out_features=100, bias=True)
  )
)

In [92]:
print(ckpt["hyper_parameters"]["feature_embeddings"]["cell_type"].embeddings.weight)
feature_embeddings = ckpt["hyper_parameters"]["feature_embeddings"]

Parameter containing:
tensor([[-2.2568, -0.7960,  0.8316,  ..., -0.9346, -1.4571, -0.5344],
        [ 1.4070,  0.2385,  0.8393,  ...,  2.0130,  2.8556, -0.6506],
        [-1.1142,  0.1809,  0.2364,  ...,  0.7844, -0.3658, -1.5056],
        ...,
        [-0.0850,  2.4505,  0.3571,  ..., -1.1968, -0.5387,  0.7276],
        [ 0.8718, -1.0743,  1.3326,  ...,  0.2681, -0.3233, -0.5216],
        [ 1.7891,  0.2539, -0.6670,  ..., -0.8340, -1.3430,  1.7463]],
       device='cuda:0', requires_grad=True)


Initializations

In [93]:
generative_model = FM(
            encoder_model=encoder_model,
            denoising_model=denoising_model,
            feature_embeddings=feature_embeddings,
            plotting_folder=None,
            in_dim=in_dim,
            size_factor_statistics=size_factor_statistics,
            encoder_type=dataset_config["encoder_type"],
            conditioning_covariate=dataset_config["conditioning_covariate"],
            model_type=denoising_model.model_type, 
            multimodal=dataset_config["multimodal"],
            is_binarized=dataset_config["is_binarized"], 
            modality_list=modality_list,
            **generative_model_config  # model_kwargs should contain the rest of the arguments
            )

# generative_model.denoising_model.embed_size_factor = True
generative_model.load_state_dict(ckpt["state_dict"])
generative_model.to("cuda")

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'encoder_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder_model'])`.
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'denoising_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['denoising_model'])`.


FM(
  (encoder_model): EncoderModel(
    (x0_from_x): ModuleDict(
      (rna): MLP(
        (net): Sequential(
          (0): Sequential(
            (0): Linear(in_features=4000, out_features=512, bias=True)
            (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ELU(alpha=1.0)
          )
          (1): Sequential(
            (0): Linear(in_features=512, out_features=300, bias=True)
            (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ELU(alpha=1.0)
          )
          (2): Linear(in_features=300, out_features=100, bias=True)
        )
      )
      (atac): MLP(
        (net): Sequential(
          (0): Sequential(
            (0): Linear(in_features=40002, out_features=1024, bias=True)
            (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ELU(alpha=1.0)
          )
          (1): Sequential(
        

# Save three copies of generated datasest 

In [79]:
adata_original = mu.read(data_path)
adata_rna = adata_original.mod["rna"]
adata_atac = adata_original.mod["atac"]
adata_rna.obs["size_factor"]=adata_rna.X.A.sum(1)
adata_atac.obs["size_factor"]=adata_atac.X.A.sum(1)
X_rna = torch.tensor(adata_rna.layers["X_counts"].todense())
X_atac = torch.tensor(adata_atac.layers["X_counts"].todense())

saving_dir = DATA_DIR / "generated" / "neurips_multimodal"

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/mudata/_core/mudata.py:577: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_common = pd.concat(


In [84]:
index_range = len(adata_rna)
num_indices = 1380

for it in tqdm(range(3)):
    X_generated_list_rna = []
    X_generated_list_atac = []
    classes_str = []
    for _ in range(10):
        
        # Generate random dummy indices without replacement
        indices = random.sample(range(index_range), num_indices)
        adata_rna_indices = adata_rna[indices]
        adata_atac_indices = adata_atac[indices]
        
        classes =  list(adata_rna_indices.obs.cell_type)
        classes_str += classes
        classes = torch.tensor([dataset.id2cov["cell_type"][c] for  c in classes])

        log_size_factors = {"rna": torch.log(torch.tensor(adata_rna_indices.layers["X_counts"].sum(1))).cuda(),
                        "atac": torch.log(torch.tensor(adata_atac_indices.layers["X_counts"].sum(1))).cuda()}
    
        X_generated = generative_model.batched_sample(batch_size=138,
                                                        repetitions=10,
                                                        n_sample_steps=2, 
                                                        covariate="cell_type", 
                                                        covariate_indices=classes, 
                                                        log_size_factor=log_size_factors)
    
        X_generated_list_rna.append(X_generated["rna"].to("cpu"))
        X_generated_list_atac.append(X_generated["atac"].to("cpu"))

    X_generated_list_rna = torch.cat(X_generated_list_rna, dim=0)
    X_generated_list_atac = torch.cat(X_generated_list_atac, dim=0)
    
    adata_generated_rna = sc.AnnData(X=sp.csr_matrix(deepcopy(X_generated_list_rna).cpu().numpy()))
    adata_generated_rna.obs["size_factor"] = adata_generated_rna.X.sum(1)
    adata_generated_rna.obs["cell_type"] = classes_str
    adata_generated_rna.obs["cell_type"] = adata_generated_rna.obs["cell_type"].astype("category")
    adata_generated_rna.layers["X_counts"] = adata_generated_rna.X.copy()
    sc.pp.normalize_total(adata_generated_rna, target_sum=10000)
    sc.pp.log1p(adata_generated_rna)
    sc.tl.pca(adata_generated_rna)
    adata_generated_rna.write_h5ad(f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/datasets/generated/neurips_multimodal/generated_cells_{it}_rna.h5ad")

    adata_generated_atac = sc.AnnData(X=sp.csr_matrix(deepcopy(X_generated_list_atac).cpu().numpy()))
    adata_generated_atac.obs["size_factor"] = adata_generated_atac.X.sum(1)
    adata_generated_atac.obs["cell_type"] = classes_str
    adata_generated_atac.obs["cell_type"] = adata_generated_atac.obs["cell_type"].astype("category")
    adata_generated_atac.layers["X_counts"] = adata_generated_atac.X.copy()
    sc.pp.normalize_total(adata_generated_atac, target_sum=10000)
    sc.pp.log1p(adata_generated_atac)
    sc.tl.pca(adata_generated_atac)
    adata_generated_atac.write_h5ad(f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/datasets/generated/neurips_multimodal/generated_cells_{it}_atac.h5ad")

100%|██████████| 3/3 [06:30<00:00, 130.24s/it]
